In [17]:
import qrcode

In [18]:
!mkdir img

In [20]:
%%bash
pip3 install git+git://github.com/ojii/pymaging.git#egg=pymaging
pip3 install git+git://github.com/ojii/pymaging-png.git#egg=pymaging-png

  Cloning git://github.com/ojii/pymaging.git to /private/var/folders/14/wzbrmqn17vlgvdhkgpqccvj40000gn/T/pip-build-aqq_t8s1/pymaging
  Running setup.py install for pymaging: started
    Running setup.py install for pymaging: finished with status 'done'
  Cloning git://github.com/ojii/pymaging-png.git to /private/var/folders/14/wzbrmqn17vlgvdhkgpqccvj40000gn/T/pip-build-yi6uwvrn/pymaging-png
  Running setup.py install for pymaging-png: started
    Running setup.py install for pymaging-png: finished with status 'done'


In [23]:
!qr --factory=pymaging "http://bit.ly/scipyla_scada_iot" > img/qr.png

[![View this Presentation Online](img/qr.png)](http://bit.ly/scipyla_scada_iot)

# From SCADA to IoT

# About me

# Where do I come from?

In [13]:
!git push origin master

Counting objects: 13, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (11/11), done.
Writing objects: 100% (13/13), 4.18 KiB | 0 bytes/s, done.
Total 13 (delta 0), reused 0 (delta 0)
To git@github.com:D3f0/talks.git
   15f0d63..7436f93  master -> master


In [15]:
!git add -u && git commit -m "Updates $(date)" && git push origin master

[master 741d146] Updates Fri May 13 16:02:12 ART 2016
 1 file changed, 52 insertions(+), 5 deletions(-)
Counting objects: 6, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 1.01 KiB | 0 bytes/s, done.
Total 6 (delta 1), reused 0 (delta 0)
To git@github.com:D3f0/talks.git
   7436f93..741d146  master -> master
